In [ ]:
""" Installation"""
import os
import sys 
import pandas as pd
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output
import numpy as np  
from datetime import datetime, time, date as dt
pd.options.mode.chained_assignment = None


# Instructions
After the dataframes of your data and locations are produced by `General.ipynb`, run this code to get a visualization.
The code needs adaptions if not all (snapchat, google photo and facebook) location dataframes have been created --> This means deleting missing parts in the "data treatment" section. 

## Data treatment 

In [ ]:
df = pd.read_pickle('General_DataFrame.pkl')
df1 = df.copy()
df1['Time'] = df1.date.apply(lambda x: str(x.time()))
df1['Date']=df1.date.apply(lambda x: str(x.date()))
df1['Type']=df1.type
df1['Label']= df1.label
df1 = df1[['Date','Time','Type','Label']]
df1

In [ ]:
df_snap = pd.read_pickle("Snapchat_location.pkl")
df_snap_loc = df_snap.copy()
df_snap_loc['time'] = df_snap_loc.date.apply(lambda x: str(x.time()))
df_snap_loc['date'] = df_snap_loc.date.apply(lambda x: str(x.date()))
df_snap_loc = df_snap_loc[['date','time','latitude','longitude']]

df_fb = pd.read_pickle("Facebook_location.pkl")
df_fb_location = df_fb.copy()
df_fb_location['time'] = df_fb_location.date.apply(lambda x: str(x.time()))
df_fb_location['date'] = df_fb_location.date.apply(lambda x: str(x.date()))
df_fb_location = df_fb_location[['date','time','latitude','longitude']]

df_google = pd.read('Google_location.pkl')
df_google_location = df_google.copy()
df_google_location['time'] = df_google_location.date.apply(lambda x: str(x.time()))
df_google_location['date']= df_google_location.date.apply(lambda x: str(x.date()))

df_location = pd.concat([df_fb_location, df_snap_loc])
df_location = pd.concat([df_location, df_google_location])
df_location

## Dash 

### Functions

In [ ]:
def get_data(get_date):
    '''
    This function return a dataframe of the selected date
    '''
    
    df = df1.loc[df1['Date']== get_date]
    
    return df 


def create_empty_table():
    '''
    This function returns an empty dataframe of the length and 'type' needed 
    '''
    table_columns = [{'name':'date', 'id':'date'},
                     {'name':'time','id':'time'},
                     {'name':'type','id':'type'},
                     {'name':'label','id':'label'}
                    ]
    
    table = dash_table.DataTable(
        id='live-table',
        data=[{}], 
        columns=table_columns,
        style_header={'backgroundColor': 'rgb(30, 30, 30)',
                     'color':'white'},
        style_cell={'textAlign': 'center',
                    'min-width':'50px',
                    'backgroundColor': '#8e8b8b'},
        style_table={'height':'300px'}
        
            ),
    
    return table

def create_table(df):
    '''
    This function returns a table of the dataframe sent
    '''
    
    table = dash_table.DataTable(
        id = 'live-table',
        columns = [{'name':i,'id':i}for i in df.columns], 
        data = df.to_dict('records'),
        style_header={'backgroundColor': 'rgb(30, 30, 30)',
                     'color':'white'},
        style_cell={
            'textAlign':'center',
            'min-width':'50px',
            'backgroundColor': '#a29f9f'},
        
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': '#8e8b8b'
        }]
    ),
    return table 
    
    
    

### Core visualization

In [ ]:
mapbox_access_token = 'Enter your mapbox token here'
ext_css = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=ext_css)
N = 100000
colors = {
    'text':'#005451',
    'plot_color': '#040404',
    'paper_color': '#040404',
}
table = create_empty_table()
app.layout = html.Div(
    style = {
        'backgroundColor': colors['paper_color'],
    },
    
    children = [
        
        html.Div([
            html.H1(children = 'Data Visualization',
                   style = {
                       'textAlign':'center',
                       'color': colors['text']
                   }),
            dcc.Graph(id='scatter_graph',
                      figure = {
                          'data':[
                              go.Scatter(
                                  x = df1.Date.unique(),
                                  y = df1.groupby('Date').Time.count(),
                                  mode = 'markers',
                                  marker = {
                                      'color' : np.random.randn(N),
                                      'colorscale':'Viridis',
                                      'line_width':1
                                  }
                              )
                          ],
                          'layout':go.Layout(
                              xaxis = {'title':'Date'},
                              yaxis = {'title':'Interactions'},
                              plot_bgcolor = colors['plot_color'],
                              paper_bgcolor = colors['paper_color'],
                              font = {
                                  'color':colors['text']
                              }
                          )
                      }),
        ]),
        
        html.Br([]),
        
        html.Div([
            html.Div(id = 'date-pick-wrapper',
                     style = {'margin':'0 auto', 'width':'49%', 'display': 'inline-block',\
                              'plot_bgcolor':colors['plot_color'],'backgroundColor': colors['paper_color']},
                     children = dcc.DatePickerSingle(
                         id='date-picker', 
                         min_date_allowed = '2011-10-20',
                         max_date_allowed = '2020-03-26',
                         initial_visible_month = '2020-01-01',
                         placeholder='Select a date :',
                         display_format = 'DD-MM-YYYY'
                     ),
                    ),
            html.Div([
                dcc.Graph(id = 'scatter_map',
                          figure = {
                              'data':[
                                  go.Scattermapbox(
                                      lat = df_location.latitude,
                                      lon = df_location.longitude,
                                      text = df_location.date,
                                      mode = 'markers',
                                      marker = {'color' : np.random.randn(N),
                                                'colorscale':'Viridis',}
                                  )
                              ],
                              'layout':go.Layout(
                                  autosize = True,
                                  showlegend = False,
                                  geo = dict(
                                      projection = dict(type = "equirectangular"),
                                  ),
                                  mapbox = dict(
                                      accesstoken = mapbox_access_token,
                                      bearing = 1,
                                      center = dict(
                                          lat = 46.216,
                                          lon = 7.002),
                                      pitch = 0,
                                      zoom = 8,
                                      style = 'dark'),
                                  paper_bgcolor = colors['paper_color']
                              ),
                          }),
            ],style={'width': '49%', 'float': 'right', 'display': 'inline-block','plot_bgcolor':colors['plot_color']})
        ],style={
            'backgroundColor': colors['paper_color'],
            'padding': '10px 5px'}),
        
        html.Br([]),
        html.Br([]),
        
        html.Div([
            html.Div(id = 'table-wrapper',
                     style={'width': '49%', 'display': 'inline-block','overflow-y': 'scroll','height':'300px',\
                            'margin':'0 auto 50px', 'plot_bgcolor':colors['plot_color'],'backgroundColor':colors['paper_color']},
                     children = table)
        ],style={
            'backgroundColor': colors['paper_color'],
            'padding': '10px 5px'}),
    ])

@app.callback(
    Output('table-wrapper', 'children'),
    [Input('scatter_graph', 'hoverData'),
     Input('date-picker', 'date'),
     Input('scatter_map','hoverData')
    ]) 

def update_table_live(hover_data, date_table, hover_map):
    ctx = dash.callback_context
    
    if not ctx.triggered:
        condition = 0
    
    else: 
        condition = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if condition != 0:
        if condition == 'scatter_graph':
            date_id = hover_data['points'][0]['x']
            df = get_data(date_id)
            table = create_table(df)
            
        elif condition == 'date-picker':
            date_id = date_table
            df = get_data(date_id)
            table = create_table(df)
        
        elif condition == 'scatter_map':
            date_id = hover_map['points'][0]['text']
            df = get_data(date_id)
            table = create_table(df)
            
    else:
        table = create_empty_table() 
        
    return table

### App

In [ ]:
if __name__ == '__main__':
    app.run_server()